In [1]:
import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *

## 1.DeepFM

In [2]:
if __name__ == "__main__":
    data = pd.read_csv('./criteo_sample.txt')

    sparse_features = ['C' + str(i) for i in range(1, 27)]
    dense_features = ['I' + str(i) for i in range(1, 14)]

    data[sparse_features] = data[sparse_features].fillna('-1', )
    data[dense_features] = data[dense_features].fillna(0, )
    target = ['label']

    # 1.Label Encoding for sparse features,and do simple Transformation for dense features
    for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
    mms = MinMaxScaler(feature_range=(0, 1))
    data[dense_features] = mms.fit_transform(data[dense_features])

    # 2.count #unique features for each sparse field,and record dense feature field name

    fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                              for feat in sparse_features] + [DenseFeat(feat, 1, )
                                                              for feat in dense_features]

    dnn_feature_columns = fixlen_feature_columns
    linear_feature_columns = fixlen_feature_columns

    feature_names = get_feature_names(
        linear_feature_columns + dnn_feature_columns)

    # 3.generate input data for model

    train, test = train_test_split(data, test_size=0.2)

    train_model_input = {name: train[name] for name in feature_names}
    test_model_input = {name: test[name] for name in feature_names}

    # 4.Define Model,train,predict and evaluate

    device = 'cpu'
    use_cuda = False
    if use_cuda and torch.cuda.is_available():
        print('cuda ready...')
        device = 'cuda:0'

    model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',
                   l2_reg_embedding=1e-5, device=device)

    model.compile("adagrad", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )
    model.fit(train_model_input,train[target].values,batch_size=32,epochs=5,verbose=2,validation_split=0.0)

    pred_ans = model.predict(test_model_input, 256)
    print("")
    print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
    print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cpu
Train on 160 samples, validate on 0 samples, 5 steps per epoch
Epoch 1/5
0s - loss:  0.6188 - binary_crossentropy:  0.6188 - auc:  0.4793
Epoch 2/5
0s - loss:  0.4964 - binary_crossentropy:  0.4964 - auc:  0.9613
Epoch 3/5
0s - loss:  0.3512 - binary_crossentropy:  0.3512 - auc:  0.9938
Epoch 4/5
0s - loss:  0.1906 - binary_crossentropy:  0.1906 - auc:  0.9936
Epoch 5/5
0s - loss:  0.1104 - binary_crossentropy:  0.1103 - auc:  0.9980

test LogLoss 0.5474
test AUC 0.5195


## 2.DIN&DIEN

- See:https://github.com/shenweichen/DeepCTR/blob/master/examples/run_din.py

In [3]:
import numpy as np
import pandas as pd
import torch

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import log_loss, roc_auc_score
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from deepctr_torch.inputs import (DenseFeat, SparseFeat, VarLenSparseFeat,get_feature_names)
from deepctr_torch.models import DIN, DIEN, DeepFM

In [4]:
train = pd.read_csv("./train.csv")
print(train.shape)
test = pd.read_csv("./test.csv")
print(test.shape)

data = pd.concat([train,test])
print(data.shape)
data = data.dropna()
print(data.shape)

(4832, 14)
(1208, 14)
(6040, 14)
(6038, 14)


In [5]:
data.head()

,userId,movieId,ratting,timestamp,last,histHighRatedMovieIds,negHistMovieIds,gender,age,occupation,zipCode,title,genres,label
0,1,48,5,978824351,1.0,1545|527|595|588|1|2355|2294|783|1566|1907,393|2562|3289|3442|2808|2259|295|1190|3321|3470,F,1,10,48067,Pocahontas (1995),Animation|Children's|Musical|Romance,1
1,2030,48,3,977809337,1.0,3928|1934|2099|3345|1013|1101|3524|3429|3751|2138,3142|1743|525|1746|3323|935|2183|3556|3343|1542,M,25,4,77345,Pocahontas (1995),Animation|Children's|Musical|Romance,0
2,4877,48,2,962767153,1.0,2078|1032|2081|616|364|2033|3034|2394|3745|3615,2354|747|3478|2638|311|1204|3370|807|29|1380,M,25,4,94703,Pocahontas (1995),Animation|Children's|Musical|Romance,0
3,2,1917,3,978300174,1.0,3418|349|1527|2353|1370|648|368|736|2002|1544,2117|2560|3480|3678|2798|1222|3872|3503|3913|3494,M,56,16,70072,Armageddon (1998),Action|Adventure|Sci-Fi|Thriller,0
4,5122,1917,5,962213528,1.0,3078|2858|2907|1584|2571|32|788|1573|2428|1580,1860|194|742|2379|1966|3397|3195|678|2240|2196,M,25,0,20009,Armageddon (1998),Action|Adventure|Sci-Fi|Thriller,1


In [6]:
sparse_features = ["userId", "movieId","gender", "age", "occupation", "zipCode"]
dense_features = ['ratting']
sequence_features = ['histHighRatedMovieIds', 'negHistMovieIds']
behavior_feature_list = ['movieId']
target = ['label']

Please check the latest version manually on https://pypi.org/project/deepctr-torch/#history


In [7]:
behavior_feature_list

['movieId']

In [8]:
def split(x, key2index):
    key_ans = x.split('|')
    for key in key_ans:
        if key not in key2index:
            # Notice : input value 0 is a special "padding",so we do not use 0 to encode valid feature for sequence input
            key2index[key] = len(key2index) + 1
    return list(map(lambda x: key2index[x], key_ans))

In [9]:
# 1.Label Encoding for sparse features,and process sequence features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
    
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

# preprocess the sequence feature
# Notice : padding=`post`
genres_key2index = {}
genres_list = list(map(lambda x: split(x, genres_key2index), data['genres'].values))
genres_length = np.array(list(map(len, genres_list)))
genres_max_len = max(genres_length)
genres_list = pad_sequences(genres_list, maxlen=genres_max_len, padding='post', )

highrate_key2index = {}
highrate_list = list(map(lambda x: split(x, highrate_key2index), data['histHighRatedMovieIds'].values))
highrate_length = np.array(list(map(len, highrate_list)))
highrate_max_len = max(highrate_length)
highrate_list = pad_sequences(highrate_list, maxlen=highrate_max_len, padding='post', )

neghist_key2index = {}
neghist_list = list(map(lambda x: split(x, neghist_key2index), data['negHistMovieIds'].values))
neghist_length = np.array(list(map(len, neghist_list)))
neghist_max_len = max(neghist_length)
neghist_list = pad_sequences(neghist_list, maxlen=neghist_max_len, padding='post', ) 

In [10]:
# 2.count #unique features for each sparse field and generate feature config for sequence feature
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique(), embedding_dim=4) for feat in sparse_features] + \
                            [DenseFeat(feat, 1, ) for feat in dense_features]

varlen_feature_columns = [VarLenSparseFeat(SparseFeat('genres', vocabulary_size=len(
    genres_key2index) + 1, embedding_dim=4), maxlen=genres_max_len, combiner='mean', length_name="seq_length")]  + \
    [VarLenSparseFeat(SparseFeat('hist_movieId', vocabulary_size=len(
    highrate_key2index) + 1, embedding_dim=4), maxlen=highrate_max_len, combiner='mean', length_name="seq_length")] + \
    [VarLenSparseFeat(SparseFeat('neg_hist_movieId', vocabulary_size=len(
    neghist_key2index) + 1, embedding_dim=4), maxlen=neghist_max_len, combiner='mean', length_name="seq_length")]


linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [11]:
feature_names

['userId',
 'movieId',
 'gender',
 'age',
 'occupation',
 'zipCode',
 'ratting',
 'genres',
 'seq_length',
 'hist_movieId',
 'neg_hist_movieId']

In [12]:
# 3.generate input data for model
split_boundary = int(data.shape[0] * 0.8)
train, test = data[:split_boundary], data[split_boundary:]

train_model_input = {name: train[name] for name in sparse_features+dense_features}
train_model_input["genres"] = genres_list[:split_boundary]
train_model_input["hist_movieId"] = highrate_list[:split_boundary]
train_model_input["neg_hist_movieId"] = neghist_list[:split_boundary]
train_model_input["seq_length"] = np.array([10 for i in range(split_boundary)])

test_model_input = {name: test[name] for name in sparse_features+dense_features}
test_model_input["genres"] = genres_list[split_boundary:]
test_model_input["hist_movieId"] = highrate_list[split_boundary:]
test_model_input["neg_hist_movieId"] = neghist_list[split_boundary:]
test_model_input["seq_length"] = np.array([10 for i in range(genres_list.shape[0]-split_boundary)])

In [13]:
test_model_input

{'userId': 0          8
 1         14
 2       2164
 3       2318
 4         15
         ... 
 1203    5960
 1204    5965
 1205    5975
 1206    6003
 1207    6005
 Name: userId, Length: 1208, dtype: int64,
 'movieId': 0       1048
 1       1622
 2       1622
 3       1622
 4       1254
         ... 
 1203     679
 1204     607
 1205     108
 1206    1600
 1207     903
 Name: movieId, Length: 1208, dtype: int64,
 'gender': 0       1
 1       1
 2       1
 3       1
 4       0
        ..
 1203    1
 1204    1
 1205    1
 1206    0
 1207    1
 Name: gender, Length: 1208, dtype: int64,
 'age': 0       2
 1       2
 2       2
 3       3
 4       3
        ..
 1203    2
 1204    6
 1205    3
 1206    0
 1207    1
 Name: age, Length: 1208, dtype: int64,
 'occupation': 0       17
 1        7
 2        7
 3        7
 4        0
         ..
 1203    15
 1204     7
 1205     1
 1206     0
 1207     4
 Name: occupation, Length: 1208, dtype: int64,
 'zipCode': 0       2106
 1        904
 2       1

In [14]:
# 4.Define Model,compile and train

device = 'cpu'
use_cuda = False
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary', device=device)

model.compile("adagrad", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )
history = model.fit(train_model_input,train[target].values,batch_size=256,epochs=5,verbose=2)

cpu
Train on 4830 samples, validate on 0 samples, 19 steps per epoch
Epoch 1/5
1s - loss:  0.6300 - binary_crossentropy:  0.6296 - auc:  0.9186
Epoch 2/5
1s - loss:  0.3885 - binary_crossentropy:  0.3877 - auc:  0.9937
Epoch 3/5
1s - loss:  0.1279 - binary_crossentropy:  0.1275 - auc:  0.9998
Epoch 4/5
1s - loss:  0.0335 - binary_crossentropy:  0.0334 - auc:  1.0000
Epoch 5/5
1s - loss:  0.0136 - binary_crossentropy:  0.0136 - auc:  1.0000


In [15]:
pred_ans = model.predict(test_model_input, 256)
print("")
for i, target_name in enumerate(target):
    print("%s test LogLoss" % target_name, round(log_loss(test[target[i]].values, pred_ans[:, i]), 4))
    print("%s test AUC" % target_name, round(roc_auc_score(test[target[i]].values, pred_ans[:, i]), 4))


label test LogLoss 0.2455
label test AUC 0.9657


In [16]:
model = DIN(linear_feature_columns, behavior_feature_list, device=device, att_weight_normalization=True)
model.compile('adagrad', 'binary_crossentropy',
              metrics=['binary_crossentropy'])
history = model.fit(train_model_input,train[target].values, batch_size=100, epochs=5, verbose=2)

cpu
Train on 4830 samples, validate on 0 samples, 49 steps per epoch
Epoch 1/5
3s - loss:  0.5720 - binary_crossentropy:  0.5696
Epoch 2/5
3s - loss:  0.1750 - binary_crossentropy:  0.1731
Epoch 3/5
3s - loss:  0.0237 - binary_crossentropy:  0.0236
Epoch 4/5
2s - loss:  0.0077 - binary_crossentropy:  0.0076
Epoch 5/5
2s - loss:  0.0041 - binary_crossentropy:  0.0041


In [106]:
pred_ans = model.predict(test_model_input, 256)
print("")
for i, target_name in enumerate(target):
    print("%s test LogLoss" % target_name, round(log_loss(test[target[i]].values, pred_ans[:, i]), 4))
    print("%s test AUC" % target_name, round(roc_auc_score(test[target[i]].values, pred_ans[:, i]), 4))


label test LogLoss 0.1496
label test AUC 0.9892


In [107]:
model = DIEN(linear_feature_columns, behavior_feature_list, use_negsampling=True, device=device, att_weight_normalization=True)
model.compile('adagrad', 'binary_crossentropy',
              metrics=['binary_crossentropy'])
history = model.fit(train_model_input,train[target].values, batch_size=100, epochs=5, verbose=2)

cpu
Train on 4830 samples, validate on 0 samples, 49 steps per epoch
Epoch 1/5
6s - loss:  0.5967 - binary_crossentropy:  0.5877
Epoch 2/5
6s - loss:  0.3385 - binary_crossentropy:  0.3301
Epoch 3/5
6s - loss:  0.2717 - binary_crossentropy:  0.2656
Epoch 4/5
6s - loss:  0.2539 - binary_crossentropy:  0.2482
Epoch 5/5
6s - loss:  0.2427 - binary_crossentropy:  0.2388


In [113]:
train[target]

,label
0,1
1,0
2,0
3,0
4,1
...,...
4827,0
4828,1
4829,1
4830,0


In [108]:
pred_ans = model.predict(test_model_input, 256)
print("")
for i, target_name in enumerate(target):
    print("%s test LogLoss" % target_name, round(log_loss(test[target[i]].values, pred_ans[:, i]), 4))
    print("%s test AUC" % target_name, round(roc_auc_score(test[target[i]].values, pred_ans[:, i]), 4))


label test LogLoss 0.3481
label test AUC 0.9991


In [19]:
data = pd.read_csv("./train.csv")
data = data.dropna()

sparse_features = ["userId", "movieId",
                   "gender", "age", "occupation", "zipCode", ]
dense_features = ['ratting']
sequence_features = ['histHighRatedMovieIds', 'negHistMovieIds']
behavior_feature_list = ['movieId']
target = ['label']

In [110]:
data['label']

0       1
1       0
2       0
3       0
4       1
       ..
1203    1
1204    1
1205    0
1206    0
1207    1
Name: label, Length: 6038, dtype: int64

In [134]:
# 1.Label Encoding for sparse features,and process sequence features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
mms = MinMaxScaler(feature_range=(0, 1))
data[dense_features] = mms.fit_transform(data[dense_features])

# preprocess the sequence feature
key2index = {}
# genres_list = list(map(split, data['genres'].values))
genres_list = list(map(lambda x: split(x, key2index), data['genres'].values))
genres_length = np.array(list(map(len, genres_list)))
max_len = max(genres_length)
# Notice : padding=`post`
genres_list = pad_sequences(genres_list, maxlen=max_len, padding='post', )

key2index1 = {}
# genres_list1 = list(map(split, data['histHighRatedMovieIds'].values))
genres_list1 = list(map(lambda x: split(x, key2index1), data['histHighRatedMovieIds'].values))
genres_length1 = np.array(list(map(len, genres_list1)))
max_len1 = max(genres_length1)
# Notice : padding=`post`
genres_list1 = pad_sequences(genres_list1, maxlen=max_len1, padding='post', )

key2index2 = {}
# genres_list2 = list(map(split, data['negHistMovieIds'].values))
genres_list2 = list(map(lambda x: split(x, key2index2), data['negHistMovieIds'].values))
genres_length2 = np.array(list(map(len, genres_list2)))
max_len2 = max(genres_length2)
# Notice : padding=`post`
genres_list2 = pad_sequences(genres_list2, maxlen=max_len2, padding='post', ) 

In [77]:
# key2index2

In [183]:
# 2.count #unique features for each sparse field and generate feature config for sequence feature

fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique(), embedding_dim=4)
                          for feat in sparse_features] + [DenseFeat(feat, 1, ) for feat in dense_features]

varlen_feature_columns = [VarLenSparseFeat(SparseFeat('genres', vocabulary_size=len(
    key2index) + 1, embedding_dim=4), maxlen=max_len, combiner='mean', length_name="seq_length")]  + \
    [VarLenSparseFeat(SparseFeat('hist_movieId', vocabulary_size=len(
    key2index1) + 1, embedding_dim=4), maxlen=max_len1, combiner='mean', length_name="seq_length")] + \
    [VarLenSparseFeat(SparseFeat('neg_movieId', vocabulary_size=len(
    key2index2) + 1, embedding_dim=4), maxlen=max_len2, combiner='mean', length_name="seq_length")]

# varlen_feature_columns += [VarLenSparseFeat(SparseFeat('genres', vocabulary_size=len(key2index) + 1, embedding_dim=4), maxlen=max_len, combiner='mean'),
#                         VarLenSparseFeat(SparseFeat('hist_movieId', vocabulary_size=len(key2index1) + 1, embedding_dim=4), maxlen=max_len1, combiner='mean'),
#                         VarLenSparseFeat(SparseFeat('negHistMovieIds', vocabulary_size=len(key2index2) + 1, embedding_dim=4), maxlen=max_len2, combiner='mean')]

linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [216]:
hist_item_id = np.array([[1, 2, 3, 0], [1, 2, 3, 0], [1, 2, 0, 0], [1, 2, 0, 0]])
hist_item_id.shape

(4, 4)

In [185]:
linear_feature_columns

[SparseFeat(name='userId', vocabulary_size=4830, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='userId', group_name='default_group'),
 SparseFeat(name='movieId', vocabulary_size=1715, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='movieId', group_name='default_group'),
 SparseFeat(name='gender', vocabulary_size=2, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='gender', group_name='default_group'),
 SparseFeat(name='age', vocabulary_size=7, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='age', group_name='default_group'),
 SparseFeat(name='occupation', vocabulary_size=21, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='occupation', group_name='default_group'),
 SparseFeat(name='zipCode', vocabulary_size=2954, embedding_dim=4, use_hash=False, dtype='int32', embedding_name='zipCode', group_name='default_group'),
 DenseFeat(name='ratting', dimension=1, dtype='float32'),
 VarLenSparseFeat(sparsefeat=SparseFea

In [223]:
# 3.generate input data for model
model_input = {name: data[name].values for name in sparse_features+dense_features}  #
model_input["genres"] = genres_list
model_input["hist_movieId"] = genres_list1
model_input["negHistMovieIds"] = genres_list2
model_input["seq_length"] = np.array([10 for i in range(4830)])

In [221]:
ss = np.array([10 for i in range(4830)])
len(ss)

4830

In [224]:
# sparse_features
model_input

{'userId': array([   0, 1590, 3882, ..., 4825, 4827, 4828]),
 'movieId': array([ 30,  30,  30, ..., 878, 205, 532]),
 'gender': array([0, 1, 1, ..., 1, 0, 0]),
 'age': array([0, 2, 2, ..., 4, 4, 4]),
 'occupation': array([10,  4,  4, ...,  7,  1,  0]),
 'zipCode': array([1349, 2078, 2685, ..., 1581, 2029,   11]),
 'ratting': array([1.  , 0.5 , 0.25, ..., 0.75, 0.5 , 0.75]),
 'genres': array([[ 1,  2,  3,  4,  0,  0],
        [ 1,  2,  3,  4,  0,  0],
        [ 1,  2,  3,  4,  0,  0],
        ...,
        [ 1,  2, 17,  3,  0,  0],
        [ 9,  7,  0,  0,  0,  0],
        [ 6,  0,  0,  0,  0,  0]], dtype=int32),
 'hist_movieId': array([[   1,    2,    3, ...,    8,    9,   10],
        [  11,   12,   13, ...,   18,   19,   20],
        [  21,   22,   23, ...,   28,   29,   30],
        ...,
        [  63,  370, 1205, ...,  333, 1232, 1440],
        [ 474,  584, 1756, ...,  672,  397,  851],
        [ 619, 1040,  229, ...,  385,  334,  203]], dtype=int32),
 'negHistMovieIds': array([[   

In [217]:
model_input['negHistMovieIds'].shape

(4830, 10)

In [225]:
# 4.Define Model,compile and train

device = 'cpu'
use_cuda = False
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary', device=device)

model.compile("adagrad", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )
history = model.fit(model_input,data[target].values,batch_size=256,epochs=2,verbose=2)

cpu
Train on 4830 samples, validate on 0 samples, 19 steps per epoch
Epoch 1/2
1s - loss:  0.6319 - binary_crossentropy:  0.6313 - auc:  0.9035
Epoch 2/2
1s - loss:  0.3779 - binary_crossentropy:  0.3772 - auc:  0.9956


In [111]:
data[target]

,label
0,1
1,0
2,0
3,0
4,1
...,...
1203,1
1204,1
1205,0
1206,0


In [117]:
# linear_feature_columns
behavior_feature_list
# model_input['negHistMovieIds'].shape

['movieId']

In [228]:
model = DIN(linear_feature_columns, behavior_feature_list, device=device, att_weight_normalization=True)
model.compile('adagrad', 'binary_crossentropy',
              metrics=['binary_crossentropy'])
history = model.fit(model_input, data[target].values, batch_size=100, epochs=5, verbose=2)

cpu
Train on 4830 samples, validate on 0 samples, 49 steps per epoch
Epoch 1/5
3s - loss:  0.4957 - binary_crossentropy:  0.4922
Epoch 2/5
3s - loss:  0.0865 - binary_crossentropy:  0.0856
Epoch 3/5
3s - loss:  0.0116 - binary_crossentropy:  0.0116
Epoch 4/5
3s - loss:  0.0047 - binary_crossentropy:  0.0047
Epoch 5/5
3s - loss:  0.0027 - binary_crossentropy:  0.0027


In [ ]:
if __name__ == "__main__":
    data = pd.read_csv("./train.csv")
    sparse_features = ["userId", "movieId",
                       "gender", "age", "occupation", "zipCode", ]
    dense_features = ['ratting']
    sequence_features = ['histHighRatedMovieIds', 'negHistMovieIds']
    behavior_feature_list = ['movieId']
    target = ['label']

    # 1.Label Encoding for sparse features,and process sequence features
    for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
    mms = MinMaxScaler(feature_range=(0, 1))
    data[dense_features] = mms.fit_transform(data[dense_features])
    
    # preprocess the sequence feature
    key2index = {}
    genres_list = list(map(split, data['genres'].values))
    genres_length = np.array(list(map(len, genres_list)))
    max_len = max(genres_length)
    # Notice : padding=`post`
    genres_list = pad_sequences(genres_list, maxlen=max_len, padding='post', )
    
    key2index1 = {}
    genres_list1 = list(map(split, data['histHighRatedMovieIds'].values))
    genres_length1 = np.array(list(map(len, genres_list1)))
    max_len1 = max(genres_length1)
    # Notice : padding=`post`
    genres_list1 = pad_sequences(genres_list1, maxlen=max_len1, padding='post', )
    
    key2index2 = {}
    genres_list2 = list(map(split, data['negHistMovieIds'].values))
    genres_length2 = np.array(list(map(len, genres_list2)))
    max_len2 = max(genres_length2)
    # Notice : padding=`post`
    genres_list2 = pad_sequences(genres_list2, maxlen=max_len2, padding='post', ) 
    

    # 2.count #unique features for each sparse field and generate feature config for sequence feature

    fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique(), embedding_dim=4)
                              for feat in sparse_features + [DenseFeat(feat, 1, )for feat in dense_features]] 

    varlen_feature_columns = [VarLenSparseFeat(SparseFeat('genres', vocabulary_size=len(
        key2index) + 1, embedding_dim=4), maxlen=max_len, combiner='mean')]  # Notice : value 0 is for padding for sequence input feature

    linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns

    feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

    # 3.generate input data for model
    model_input = {name: data[name] for name in sparse_features}  #
    model_input["genres"] = genres_list

    # 4.Define Model,compile and train

    device = 'cpu'
    use_cuda = True
    if use_cuda and torch.cuda.is_available():
        print('cuda ready...')
        device = 'cuda:0'

    model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)

    model.compile("adam", "mse", metrics=['mse'], )
    history = model.fit(model_input,data[target].values,batch_size=256,epochs=10,verbose=2,validation_split=0.2)